<a href="https://colab.research.google.com/github/robert-sturrock/clue_boardgame_solver/blob/main/Clue_board_game_solver_explaination_%2B_game_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clue boardgame assistant

Aim is to create a tool that successfully performs most of the reasoning for you in Clue.

In [17]:
#@title Set up code here{display-mode: "form"} 
import numpy as np
import matplotlib as m
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import copy

import re as re
import pandas as pd
import unicodedata 
import requests
import seaborn as sns
%matplotlib inline

## Create base clue object

### Hand object

Set up what each players hand can contain.

In [18]:
class Hand:

  def __init__(self):
    # add base features:
    self.characters = {'mustard':[None,[]], 
                       'plum':[None,[]], 
                       'green':[None,[]], 
                       'peacock':[None,[]], 
                       'scarlet':[None,[]], 
                       'white':[None,[]]
                       }
    self.weapons = {'knife':[None,[]],
                    'candlestick':[None,[]], 
                    'revolver':[None,[]], 
                    'rope':[None,[]], 
                    'lead pipe':[None,[]], 
                    'wrench':[None,[]]
                    }
    self.rooms = {'hall':[None,[]],
                  'lounge':[None,[]], 
                  'dining room':[None,[]], 
                  'kitchen':[None,[]],
                  'ballroom':[None,[]], 
                  'conservatory':[None,[]],
                  'billiard room':[None,[]],
                  'library':[None,[]],
                  'study':[None,[]],
                  }
    self.moves = []

  def initial_cards(self, initial_cards):
    # use this to initialize your hand (where you have full knowledge of cards)
    for group in [self.characters, self.weapons, self.rooms]:  
      for (key,val) in group.items():
        if key in initial_cards:
          group[key][0] = 1
        else:
          group[key][0] = 0

  def round(self, move):
    self.moves.append(move)

### Clue_Game Object

Holds multiple hands (ie your representation, as a player, of what you and everyone else is holding). It will also hold the "move" method that runs eliminations and so on. 

If you get to a place where a card has been eliminated from every other players hand then you can conclude that that card is in the envelope (ie was involved in the murder). 



In [19]:
class Clue_Game:

  def __init__(self, players, initial_cards):
    # always list yourself as first  **TO DO**: maybe use a specific name like "self" and then name opponents
    self.players = players
    self.moves = []
    self.hands = []
    self.partial_info_number = 0
    
    # last turn values
    self.hands_prev = []
    self.moves_prev = []
    self.partial_info_number_prev = []

    # initiate Hands:
    for player in self.players:
      hand = {'player':player, 'hand':Hand()}
      
      if player == 'self':
        hand['hand'].initial_cards(initial_cards)

      if player != 'self':
        for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
          for (key,val) in group.items():
            if key in initial_cards:
              group[key][0] = 0   
                   
      self.hands.append(hand)
  
  def players_involved(self, player, responder):
    if responder == None:
      return [p for p in self.players if p != player]
    else:
      start, end = [self.players.index(x) for x in [player, responder]]
      if start <= end:
          return self.players[start:end+1]
      else:
          return self.players[start:] + self.players[:end+1]
  
  def eliminate_non_response(self, player, person, weapon, room):
    # for a specific player eliminate a person and weapon
    for hand in self.hands:
        if hand['player'] == player:
          hand['hand'].characters[person][0] = 0
          hand['hand'].weapons[weapon][0] = 0
          hand['hand'].rooms[room][0] = 0
  
  def player_has_card(self, card, responder):
    # identify if we know that a responder (with hidden response) is holding one 
    # of the cards that they have been asked about.
    has_card = 0
    for hand in self.hands:
      if hand['player'] == responder:
        for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
          for (key,val) in group.items():
            if key in [card] and group[key][0] == 1:
              has_card += 1
    return has_card

  def record_response(self, player, person, weapon, room, responder=None, response=None):

    # record what the response is  if you are told
    if response not in [None,'hidden']:
      for hand in self.hands:
        if hand['player'] == responder:
          for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]: 
              for (key,val) in group.items():
                if key == response:
                  group[key][0] = 1
      
        # eliminate the option for everyone else
        if hand['player'] != responder:
          for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
            for (key,val) in group.items():
              if key == response:
                group[key][0] = 0
                group[key][1] = []

    # record the clues as to the response if you don't see it 
    # partial_info_number indicates what options are possibilities
    if response == 'hidden':
      has_card = 0
      for card in [person, weapon, room]:
        has_card += self.player_has_card(card, responder)
      # if player doesn't have any of the cards in their hand then we can record a partial info number
      if has_card == 0:
        for hand in self.hands:
          if hand['player'] == responder:
            for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
              for (key,val) in group.items():
                if key in [person, weapon, room] and group[key][0] not in [0,1]:
                  group[key][1].append(tmp.partial_info_number)

        
      # increment info number
      self.partial_info_number += 1

  def check_for_eliminations(self):
    last_item_found = None
    # look for cases where partial_info_numbers cancel out revealing true answer
    hand_checks = 1
    while hand_checks > 0:
      for hand in self.hands:
        # for each partial info number look at each hand and count up instances
        for number in range(self.partial_info_number):
          check_pin_count = 0 

          for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
            for (key,val) in group.items():
              if number in group[key][1]:
                check_pin_count += 1
        
          # if instances = 1 then indicate player has that card
          # *Note*: we don't remove partial info number for a 1, as there is
          # a possibility that player doesn't have the other card
          # (this will only happen once per partial_info_number)
          if check_pin_count == 1:
            for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
              for (key,val) in group.items():
                if number in group[key][1]:
                  group[key][0] = 1
                  group[key][1].remove(number)
                  # record card found and player it belongs to
                  last_item_found = key
                  player_item_found = hand['player']
                  hand_checks += 1

      # eliminate newly found card/item from other hands
      for hand in self.hands:         
        for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
          for (key,val) in group.items():
            if key == last_item_found and hand['player'] != player_item_found:
              group[key][0] = 0
              group[key][1] = []

      hand_checks -=1
 
  def move(self, player, person, weapon, room, responder=None, response=None):
    
    # back up previous hand states
    self.hands_prev = copy.deepcopy(self.hands)
    self.moves_prev = copy.deepcopy(self.moves)
    self.partial_info_number_prev = copy.copy(self.partial_info_number)

    # global  = list(self.moves)

    # record move
    rec = {'player':player, 'person':person, 'weapon':weapon, 'room':room,
           'responder':responder, 'response':response}
    self.moves.append(rec)

    # establish players involved
    players_involved = self.players_involved(player, responder)

    # eliminate options on players that didn't play
    no_match_players = list(set(players_involved) - set([player,responder]))
    for p in no_match_players:
      self.eliminate_non_response(p, person, weapon, room)

    # record information learned from the response or lack of
    self.record_response(player, person, weapon, room, responder, response)

    # look for possible eliminations
    self.check_for_eliminations()

  def hand_status(self):
    '''Get a dataframe of current hand information for all players'''
    # records 
    hand_records = []
    partial_info_records = []
    for hand in self.hands:
      player = hand['player']
      hc = pd.DataFrame(hand['hand'].characters)
      hw = pd.DataFrame(hand['hand'].weapons)
      hr = pd.DataFrame(hand['hand'].rooms)
      # store card knowledge and partial knowledge
      hand_col = pd.concat([hc,hw,hr],axis=1).loc[0]
      partial_info_col = pd.concat([hc,hw,hr],axis=1).loc[1]
      hand_col['player'] = player
      partial_info_col['player'] = player
      # record info for player
      hand_records.append(hand_col)
      partial_info_records.append(partial_info_col)
  
    return [pd.DataFrame(hand_records).set_index('player').transpose(),
            pd.DataFrame(partial_info_records).set_index('player').transpose()]

  def moves_hist(self):
    '''readable dataframe of moves'''
    return pd.DataFrame(self.moves)

  def remove_last_move(self):
    '''Use this to correct an incorrect move'''
    self.moves = copy.deepcopy(self.moves_prev)
    self.hands = copy.deepcopy(self.hands_prev)
    self.partial_info_number = copy.copy(self.partial_info_number_prev)
    


## The basics / example game: 

There are a few key methods you will use to make use of the solver: 
* .move(player, person, weapon, room, responder, response): 
    * player: who is asking / accusing this round
    * person: which character is being accused (eg. 'plum')
    * weapon: weapon that was used (eg. 'knife')
    * room: place it happened (eg. 'hall')
    * responder: which player responded, can be 'None' if no one has a matching card
    * response: either the card shown if you are the asker (eg. 'hall') or 'hidden' if you do not see what is shown
* .hand_status(): will give you two print outs. The first covers what is known for certain about the hands (0 and 1 for has or does not have, None / NaN for no information known yet). The second gives you 'partial info' about what is known (ie if you saw *player 1* ask about ('plum','knife','hall') and you saw *player 2* respond then it will place a number next to 'plum','knife',and 'hall'. If you learn that 'plum' is in *player 1*'s hand and 'knife' is in *player 3*'s hand then it will conclude that 'hall' must have been in *player 2*'s hand. These eliminations are some of the main value add of the solver. 

* remove_last_move(): removes the last move (helpful if you made a typo)

* moves_hist(): shows you a record of all moves played in a game.

Here is how things look:

First you create the game like this:
* **players**: the names you want to use, make sure players are in the order they play in (this is used in ruling out cards from players hands)
* **initial cards**: what were your initial cards

In [20]:
# Create the game:
tmp = Clue_Game(players = ['self','dad','mom','isabel'], 
                initial_cards=['white','peacock','green','conservatory'])

Show all the information you know: (less nice visual)

In [21]:
tmp.hand_status()

[player         self  dad  mom  isabel
 mustard         0.0  NaN  NaN     NaN
 plum            0.0  NaN  NaN     NaN
 green           1.0  0.0  0.0     0.0
 peacock         1.0  0.0  0.0     0.0
 scarlet         0.0  NaN  NaN     NaN
 white           1.0  0.0  0.0     0.0
 knife           0.0  NaN  NaN     NaN
 candlestick     0.0  NaN  NaN     NaN
 revolver        0.0  NaN  NaN     NaN
 rope            0.0  NaN  NaN     NaN
 lead pipe       0.0  NaN  NaN     NaN
 wrench          0.0  NaN  NaN     NaN
 hall            0.0  NaN  NaN     NaN
 lounge          0.0  NaN  NaN     NaN
 dining room     0.0  NaN  NaN     NaN
 kitchen         0.0  NaN  NaN     NaN
 ballroom        0.0  NaN  NaN     NaN
 conservatory    1.0  0.0  0.0     0.0
 billiard room   0.0  NaN  NaN     NaN
 library         0.0  NaN  NaN     NaN
 study           0.0  NaN  NaN     NaN, player        self dad mom isabel
 mustard         []  []  []     []
 plum            []  []  []     []
 green           []  []  []     []
 p

Just show information you are certain about:

In [22]:
tmp.hand_status()[0]

player,self,dad,mom,isabel
mustard,0.0,NaN,NaN,NaN
plum,0.0,NaN,NaN,NaN
green,1.0,0.0,0.0,0.0
peacock,1.0,0.0,0.0,0.0
scarlet,0.0,NaN,NaN,NaN
white,1.0,0.0,0.0,0.0
knife,0.0,NaN,NaN,NaN
candlestick,0.0,NaN,NaN,NaN
revolver,0.0,NaN,NaN,NaN
rope,0.0,NaN,NaN,NaN


Game starts, enter a move:

The moves below were a full game in Clue.

In [23]:
tmp.move(player='dad',person='scarlet',weapon='lead pipe',room='lounge',
         responder='mom',response='hidden')

In [24]:
tmp.move(player='mom',person='plum',weapon='knife',room='conservatory',
         responder='self',response='conservatory')

In [25]:
tmp.move(player='isabel',person='plum',weapon='knife',room='billiard room',
         responder='dad',response='hidden')

In [26]:
tmp.move(player='self',person='white',weapon='lead pipe',room='study',
         responder='dad',response='lead pipe')

In [27]:
tmp.move(player='dad',person='scarlet',weapon='lead pipe',room='conservatory',
         responder='self',response='conservatory')

In [28]:
tmp.move(player='mom',person='scarlet',weapon='revolver',room='lounge',
         responder='dad',response='hidden')

In [29]:
tmp.move(player='isabel',person='scarlet',weapon='revolver',room='ballroom',
         responder='dad',response='hidden')

In [30]:
tmp.move(player='self',person='scarlet',weapon='revolver',room='study',
         responder='dad',response='scarlet')

In [31]:
tmp.move(player='dad',person='mustard',weapon='candlestick',room='ballroom',
         responder='mom',response='hidden')

In [32]:
tmp.move(player='mom',person='plum',weapon='knife',room='conservatory',
         responder='self',response='conservatory')

In [33]:
tmp.move(player='isabel',person='white',weapon='revolver',room='hall',
         responder='self',response='white')

In [34]:
tmp.move(player='self',person='green',weapon='revolver',room='ballroom',
         responder='mom',response='ballroom')

In [35]:
tmp.move(player='dad',person='mustard',weapon='rope',room='ballroom',
         responder='mom',response='hidden')

In [36]:
tmp.move(player='mom',person='plum',weapon='rope',room='dining room',
         responder='isabel',response='hidden')

In [37]:
tmp.move(player='isabel',person='green',weapon='candlestick',room='dining room',
         responder='self',response='green')

In [38]:
tmp.move(player='self',person='green',weapon='candlestick',room='dining room',
         responder='mom',response='candlestick')

In [39]:
tmp.move(player='dad',person='peacock',weapon='wrench',room='ballroom',
         responder='mom',response='hidden')

In [40]:
tmp.move(player='mom',person='plum',weapon='candlestick',room='dining room',
         responder=None,response=None)

In [41]:
tmp.move(player='isabel',person='peacock',weapon='lead pipe',room='ballroom',
         responder='self',response='peacock')

In [42]:
tmp.move(player='self',person='peacock',weapon='revolver',room='dining room',
         responder=None,response=None)

If you look at what we know at this point you'll see two options now can't be in anyone's hand: 
'dining room' and 'revolver:

Moreover, because 'mom' got a non response a few turns ago on 'plum' - and I knew she had 'candlestick' already I took a guess that she didn't have 'plum' and made a formal accusation at this point - which was a bit of a gamble but worked out.

In [44]:
tmp.hand_status()[0]

player,self,dad,mom,isabel
mustard,0.0,NaN,NaN,NaN
plum,0.0,0.0,NaN,0.0
green,1.0,0.0,0.0,0.0
peacock,1.0,0.0,0.0,0.0
scarlet,0.0,1.0,0.0,0.0
white,1.0,0.0,0.0,0.0
knife,0.0,NaN,NaN,0.0
candlestick,0.0,0.0,1.0,0.0
revolver,0.0,0.0,0.0,0.0
rope,0.0,NaN,NaN,NaN


Show all the took place in this game:

In [45]:
tmp.moves_hist()

,player,person,weapon,room,responder,response
0,dad,scarlet,lead pipe,lounge,mom,hidden
1,mom,plum,knife,conservatory,self,conservatory
2,isabel,plum,knife,billiard room,dad,hidden
3,self,white,lead pipe,study,dad,lead pipe
4,dad,scarlet,lead pipe,conservatory,self,conservatory
5,mom,scarlet,revolver,lounge,dad,hidden
6,isabel,scarlet,revolver,ballroom,dad,hidden
7,self,scarlet,revolver,study,dad,scarlet
8,dad,mustard,candlestick,ballroom,mom,hidden
9,mom,plum,knife,conservatory,self,conservatory


Here's the partial info we had

In [46]:
tmp.hand_status()[1]

player,self,dad,mom,isabel
mustard,[],[],[4],[]
plum,[],[1],[],[]
green,[],[],[],[]
peacock,[],[],[],[]
scarlet,[],"[2, 3]",[],[]
white,[],[],[],[]
knife,[],[1],[],[]
candlestick,[],[],[4],[]
revolver,[],"[2, 3]",[],[]
rope,[],[],[],[6]


## Your game here

See "the basics / example game" for more details.

In [ ]:
# Create the game:
# write in your player names
# write in your starting cards
tmp = Clue_Game(players = ['player 1','player 2','player 3'], initial_cards=[])

Record moves

In [ ]:
tmp.move(player=, person=, weapon=, responder=, response=)